In [ ]:
import torch


# Calculating Index in a 2D image 

### Row Major Layout

```c
int row = blockIdx.y * blockDim.y + threadIdx.y;
int col = blockIdx.x * blockDim.x + threadIdx.x;
int plane = blockIdx.z * blockDim.z + threadIdx.z;

int offset = plane * m * n + row * m + col;
```

### Column Major Layout

### BLAS

Level 1: Vector-vector operations
Level 2: Matrix-vector operations
Level 3: Matrix-matrix operations

